In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def criar_cnn(input_shape):
    # 1. Definir a arquitetura da CNN
    model = models.Sequential()

    # Primeira camada convolutiva
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling1D(pool_size=2))

    # Segunda camada convolutiva
    model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))

    # Flatten para conectar às camadas totalmente conectadas
    model.add(layers.Flatten())

    # Camada totalmente conectada
    model.add(layers.Dense(64, activation='relu'))

    # Camada de saída com função de ativação sigmoid (classificação binária)
    model.add(layers.Dense(1, activation='sigmoid'))

    # 2. Compilar o modelo
    model.compile(optimizer='adam', 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])

    # 3. Resumo do modelo
    model.summary()
    
    return model


In [ ]:
def treinar_avaliar_cnn(model, X_train, y_train, X_val, y_val, X_test, y_test, epochs=20, batch_size=32):
    # Treinamento do modelo
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

    # Avaliação do modelo
    test_loss, test_acc = model.evaluate(X_test, y_test)
    print(f'Test accuracy: {test_acc}')
    
    return history


In [ ]:
input_shape = (X_train.shape[1], 1)  # Exemplo de input shape
model = criar_cnn(input_shape)


history = treinar_avaliar_cnn(model, X_train, y_train, X_val, y_val, X_test, y_test)
